Program that prompts user for song data and then adds it to database

In [ ]:
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('GuitarSongs')

inputing = input("Would you like to add a new song to the list? y/n ")
while inputing != "n":
    strings = int(input("How many strings does the guitar have? "))
    drop = "t" if input("Is it in drop tuning? y/n ") != "n" else "f"
    notes = ["A", "A#", "B", "C", "C#", "D", "Eb", "E", "F", "Gb", "G", "Ab"]
    note = notes[int(input("Input number corressponding to the note of the tuning 1-12 \n " + str(notes)))-1]
    name = input("What is the name of the song: ")
    album = input("Which album is it on: ")
    artist = input("Who is the artist: ")
    table.put_item(
        Item = {
          "Tuning": f"strings#{strings}#drop#{drop}#{note}",
          "SongName": name,
          "Album": album,
          "Artist": artist
        }
    )
    inputing = input("Would you like to add a new song to the list? y/n ")


Basic Querying of a table (dev-blackout-monitor-minutes)
Asking user for a date and an hour, and displaying how long power is present/absent for

In [6]:
import boto3
from boto3.dynamodb.conditions import Key
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('dev-blackout-monitor-minutes')

# date = input("Enter the date for which you want to see the data (format yyyy-mm-dd)")
# h = input("Enter for which hour would you like to see the data")
# hour = h if len(str(h))==2 else ("0" + h)  

date = "2025-07-13"
hour = "16"


response = table.query(
    IndexName = "sk_status-index",
    KeyConditionExpression=Key("pk_status").eq("device#alex_rpi#sensor#4#status#done") & Key("ts").begins_with(f"{date}T{str(hour)}")
)
# print(response)

on_c = 0
off_c = 0 
for each in response['Items']:
    if each['reading'] == 0: off_c += 1
    else: on_c += 1

print(f"Power present for {on_c} minutes and absent for {off_c}")

Power present for 0 minutes and absent for 60


Now with scanning

In [14]:
import boto3
from boto3.dynamodb.conditions import Key
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('dev-blackout-monitor-minutes')

# date = input("Enter the date for which you want to see the data (format yyyy-mm-dd)")
# h1 = input("Enter from which hour would you like to see the data")
# h2 = input("Enter until which hour would you like to see the data")
# hour1 = h1 if len(str(h1))==2 else ("0" + h1)
# hour2 = h2 if len(str(h2))==2 else ("0" + h2)  

date = "2025-06-21"
hour1 = "19"
hour2 = "20"

scan_args = {
    "FilterExpression" : Key("ts").begins_with(f"{date}T{hour1}"),
    "ProjectionExpression": "#t, #r",
    "ExpressionAttributeNames": {"#t": "ts", "#r": "reading"}
}
response = table.scan(**scan_args)
# print(response)

on_c = 0
off_c = 0 
for each in response['Items']:
    if each['reading'] == 0: off_c += 1
    else: on_c += 1

print(f"Power present for {on_c} minutes and absent for {off_c}")

Power present for 60 minutes and absent for 0
